In [21]:
# Q1) YoY monthly growth (Apr-Nov 2024 vs Apr-Nov 2025)
# Combined product logic: STX INSTA SHAMPOO + STREAX INSTA SHAMPOO
# Output: ONLY 2 tables -> 12-ML and 18-ML

from pathlib import Path
import numpy as np
import pandas as pd
from IPython.display import display

candidate_dirs = [Path('../DATA'), Path('DATA'), Path.cwd() / 'DATA']
data_dir = next((p for p in candidate_dirs if p.exists()), None)
if data_dir is None:
    raise FileNotFoundError("DATA folder not found. Expected ../DATA or DATA")

files = sorted(data_dir.glob('*.parquet'))
if not files:
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

use_cols = ['Date', 'SalesValue_atBasicRate', 'Quantity', 'Subcategory', 'Sizes']
df = pd.concat((pd.read_parquet(f, columns=use_cols) for f in files), ignore_index=True)

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date']).copy()

# Normalize text
df['Subcategory'] = (
    df['Subcategory'].astype(str).str.upper().str.replace(r'\s+', ' ', regex=True).str.strip()
)
df['Sizes'] = (
    df['Sizes'].astype(str).str.upper().str.replace(' ', '').str.strip()
)

# Keep only the two "same" subcategories and 12/18 ml
target_subcats = {'STX INSTA SHAMPOO', 'STREAX INSTA SHAMPOO'}
target_sizes = {'12-ML', '18-ML'}

df = df[df['Subcategory'].isin(target_subcats) & df['Sizes'].isin(target_sizes)].copy()

def yoy_table_for_size(frame):
    w = frame.copy()
    w['year'] = w['Date'].dt.year
    w['month'] = w['Date'].dt.month
    w = w[w['year'].isin([2024, 2025]) & w['month'].between(4, 11)]

    monthly = (
        w.groupby(['year', 'month'], as_index=False)
         .agg(sales_value=('SalesValue_atBasicRate', 'sum'),
              quantity=('Quantity', 'sum'))
    )

    piv = monthly.pivot(index='month', columns='year', values=['sales_value', 'quantity'])
    months = list(range(4, 12))
    labels = {4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov'}

    out = pd.DataFrame({'Month': [labels[m] for m in months]})

    for metric, prefix in [('sales_value', 'Sales'), ('quantity', 'Volume')]:
        y24 = piv.get((metric, 2024), pd.Series(index=months, dtype=float)).reindex(months)
        y25 = piv.get((metric, 2025), pd.Series(index=months, dtype=float)).reindex(months)
        yoy = np.where(y24.fillna(0).values != 0, (y25.values - y24.values) / y24.values * 100, np.nan)

        out[f'{prefix}_2024'] = y24.values
        out[f'{prefix}_2025'] = y25.values
        out[f'{prefix}_YoY_%'] = yoy

    out['Sales_2024'] = out['Sales_2024'].round(2)
    out['Sales_2025'] = out['Sales_2025'].round(2)
    out['Volume_2024'] = out['Volume_2024'].round(0)
    out['Volume_2025'] = out['Volume_2025'].round(0)
    out['Sales_YoY_%'] = out['Sales_YoY_%'].round(2)
    out['Volume_YoY_%'] = out['Volume_YoY_%'].round(2)

    return out

for size in ['12-ML', '18-ML']:
    part = df[df['Sizes'] == size].copy()  # combined STX + STREAX
    tbl = yoy_table_for_size(part)

    print(f"\n=== INSTA SHAMPOO COMBINED (STX + STREAX) | {size} ===")
    print(tbl.to_string(index=True))

    out_file = Path(f"yoy_insta_combined_{size.lower()}.csv")
    tbl.to_csv(out_file, index=False)
    print(f"Saved: {out_file.resolve()}")



=== INSTA SHAMPOO COMBINED (STX + STREAX) | 12-ML ===
  Month  Sales_2024   Sales_2025  Sales_YoY_%  Volume_2024  Volume_2025  Volume_YoY_%
0   Apr  2784152.50  10885588.39       290.98     245580.0     958622.0        290.35
1   May  3708496.81   3148684.03       -15.10     327035.0     277412.0        -15.17
2   Jun  3391510.84   4535698.35        33.74     298679.0     399621.0         33.80
3   Jul  3320482.23   8810406.77       165.34     292422.0     776399.0        165.51
4   Aug  4106699.64   3577602.22       -12.88     364412.0     315394.0        -13.45
5   Sep  3692030.11  11292128.25       205.85     352283.0     994975.0        182.44
6   Oct  3281060.09   4126506.16        25.77     312891.0     363712.0         16.24
7   Nov  3556759.85  11262209.99       216.64     337189.0     992433.0        194.33
Saved: C:\Users\abqua\Desktop\HRI app\jupyter\yoy_insta_combined_12-ml.csv

=== INSTA SHAMPOO COMBINED (STX + STREAX) | 18-ML ===
  Month   Sales_2024   Sales_2025  Sales_

In [27]:
from pathlib import Path
import pandas as pd
from IPython.display import display

# -------- Load --------
candidate_dirs = [Path('../DATA'), Path('DATA'), Path.cwd() / 'DATA']
data_dir = next((p for p in candidate_dirs if p.exists()), None)
if data_dir is None:
    raise FileNotFoundError("DATA folder not found. Expected ../DATA or DATA")

files = sorted(data_dir.glob('*.parquet'))
if not files:
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

use_cols = ['Subcategory', 'Sizes', 'SalesValue_atBasicRate', 'Quantity']
df = pd.concat((pd.read_parquet(f, columns=use_cols) for f in files), ignore_index=True)

# -------- Clean --------
df['Subcategory'] = (
    df['Subcategory'].astype(str).str.upper().str.replace(r'\s+', ' ', regex=True).str.strip()
)
df['Sizes'] = df['Sizes'].astype(str).str.upper().str.replace(' ', '').str.strip()

# -------- Scope: combined subcategories (all sizes) --------
target_subcats = ['STX INSTA SHAMPOO', 'STREAX INSTA SHAMPOO']
scope_all_sizes = df[df['Subcategory'].isin(target_subcats)].copy()

# Denominator = total of these 2 subcategories across all sizes
total_scope_sales = float(scope_all_sizes['SalesValue_atBasicRate'].sum())
total_scope_qty = float(scope_all_sizes['Quantity'].sum())

# -------- Numerator: only 12-ML and 18-ML within same scope --------
target_sizes = ['12-ML', '18-ML']
sku_df = scope_all_sizes[scope_all_sizes['Sizes'].isin(target_sizes)].copy()

out = (
    sku_df.groupby('Sizes', as_index=False)
    .agg(
        sales_value=('SalesValue_atBasicRate', 'sum'),
        quantity=('Quantity', 'sum')
    )
    .rename(columns={'Sizes': 'SKU_Size'})
)

out['total_scope_sales'] = total_scope_sales
out['total_scope_qty'] = total_scope_qty
out['value_contribution_%'] = (out['sales_value'] / total_scope_sales * 100).round(2)
out['volume_contribution_%'] = (out['quantity'] / total_scope_qty * 100).round(2)

# formatting
out['sales_value'] = out['sales_value'].round(2)
out['quantity'] = out['quantity'].round(0)
out['total_scope_sales'] = out['total_scope_sales'].round(2)
out['total_scope_qty'] = out['total_scope_qty'].round(0)

display(out.sort_values('SKU_Size').reset_index(drop=True))


,SKU_Size,sales_value,quantity,total_scope_sales,total_scope_qty,value_contribution_%,volume_contribution_%
0,12-ML,1.047080e+08,9370247.0,4.924663e+08,34683903.0,21.26,27.02
1,18-ML,3.217894e+08,20949030.0,4.924663e+08,34683903.0,65.34,60.40


In [23]:
# Absolute totals (no %) for both 12-ML and 18-ML
# Combined: STX INSTA SHAMPOO + STREAX INSTA SHAMPOO
# Within Brand = STREAX

from pathlib import Path
import pandas as pd
from IPython.display import display

candidate_dirs = [Path('../DATA'), Path('DATA'), Path.cwd() / 'DATA']
data_dir = next((p for p in candidate_dirs if p.exists()), None)
if data_dir is None:
    raise FileNotFoundError("DATA folder not found. Expected ../DATA or DATA")

files = sorted(data_dir.glob('*.parquet'))
if not files:
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

use_cols = ['Brand', 'Subcategory', 'Sizes', 'SalesValue_atBasicRate', 'Quantity']
df = pd.concat((pd.read_parquet(f, columns=use_cols) for f in files), ignore_index=True)

df['Brand'] = df['Brand'].astype(str).str.upper().str.strip()
df['Subcategory'] = df['Subcategory'].astype(str).str.upper().str.replace(r'\s+', ' ', regex=True).str.strip()
df['Sizes'] = df['Sizes'].astype(str).str.upper().str.replace(' ', '').str.strip()

brand_df = df[df['Brand'] == 'STREAX'].copy()

target_subcats = ['STX INSTA SHAMPOO', 'STREAX INSTA SHAMPOO']
target_sizes = ['12-ML', '18-ML']

scope = brand_df[
    brand_df['Subcategory'].isin(target_subcats) &
    brand_df['Sizes'].isin(target_sizes)
].copy()

abs_totals = (
    scope.groupby('Sizes', as_index=False)
    .agg(
        total_sales_value=('SalesValue_atBasicRate', 'sum'),
        total_volume=('Quantity', 'sum')
    )
    .rename(columns={'Sizes': 'SKU_Size'})
    .sort_values('SKU_Size')
    .reset_index(drop=True)
)

abs_totals['total_sales_value'] = abs_totals['total_sales_value'].round(2)
abs_totals['total_volume'] = abs_totals['total_volume'].round(0)

display(abs_totals)

# Add contribution % (value + volume) and save CSV

brand_sales_total = scope['SalesValue_atBasicRate'].sum()
brand_volume_total = scope['Quantity'].sum()

abs_totals['value_contribution_%'] = (
    (abs_totals['total_sales_value'] / brand_sales_total * 100)
    if brand_sales_total != 0 else 0
)
abs_totals['volume_contribution_%'] = (
    (abs_totals['total_volume'] / brand_volume_total * 100)
    if brand_volume_total != 0 else 0
)

abs_totals['value_contribution_%'] = abs_totals['value_contribution_%'].round(2)
abs_totals['volume_contribution_%'] = abs_totals['volume_contribution_%'].round(2)

# Optional: reorder columns
abs_totals = abs_totals[
    ['SKU_Size', 'total_sales_value', 'total_volume', 'value_contribution_%', 'volume_contribution_%']
]

display(abs_totals)

out_file = Path("insta_shampoo_12ml_18ml_abs_totals_with_contribution.csv")
abs_totals.to_csv(out_file, index=False)
print(f"Saved: {out_file.resolve()}")



,SKU_Size,total_sales_value,total_volume
0,12-ML,9.344580e+07,8377814.0
1,18-ML,3.078042e+08,20099264.0


,SKU_Size,total_sales_value,total_volume,value_contribution_%,volume_contribution_%
0,12-ML,9.344580e+07,8377814.0,23.29,29.42
1,18-ML,3.078042e+08,20099264.0,76.71,70.58


Saved: C:\Users\abqua\Desktop\HRI app\jupyter\insta_shampoo_12ml_18ml_abs_totals_with_contribution.csv


In [6]:
# Within GT: outlet classification mix for 12-ML and 18-ML
# Scope = Brand STREAX, Subcategory in {STX INSTA SHAMPOO, STREAX INSTA SHAMPOO}
# Shows:
# 1) Detailed Final_Outlet_Classification mix
# 2) WH vs Others split (sales % and volume %)

from pathlib import Path
import pandas as pd
from IPython.display import display

candidate_dirs = [Path('../DATA'), Path('DATA'), Path.cwd() / 'DATA']
data_dir = next((p for p in candidate_dirs if p.exists()), None)
if data_dir is None:
    raise FileNotFoundError("DATA folder not found. Expected ../DATA or DATA")

files = sorted(data_dir.glob('*.parquet'))
if not files:
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

use_cols = [
    'Brand', 'Subcategory', 'Sizes', 'Outlet_Type', 'Final_Outlet_Classification',
    'SalesValue_atBasicRate', 'Quantity'
]
df = pd.concat((pd.read_parquet(f, columns=use_cols) for f in files), ignore_index=True)

# Normalize text fields
df['Brand'] = df['Brand'].astype(str).str.upper().str.strip()
df['Subcategory'] = df['Subcategory'].astype(str).str.upper().str.replace(r'\s+', ' ', regex=True).str.strip()
df['Sizes'] = df['Sizes'].astype(str).str.upper().str.replace(' ', '').str.strip()
df['Outlet_Type'] = df['Outlet_Type'].astype(str).str.upper().str.strip()
df['Final_Outlet_Classification'] = df['Final_Outlet_Classification'].astype(str).str.upper().str.strip()

target_subcats = ['STX INSTA SHAMPOO', 'STREAX INSTA SHAMPOO']
target_sizes = ['12-ML', '18-ML']

scope = df[
    (df['Brand'] == 'STREAX') &
    (df['Subcategory'].isin(target_subcats)) &
    (df['Sizes'].isin(target_sizes)) &
    (df['Outlet_Type'] == 'GT')
].copy()

for size in target_sizes:
    part = scope[scope['Sizes'] == size].copy()

    print(f"\n=== GT Outlet Classification Mix | INSTA SHAMPOO (STX+STREAX) | {size} ===")

    # 1) Detailed classification mix
    detail = (
        part.groupby('Final_Outlet_Classification', as_index=False)
            .agg(
                sales_value=('SalesValue_atBasicRate', 'sum'),
                quantity=('Quantity', 'sum')
            )
            .sort_values('sales_value', ascending=False)
            .reset_index(drop=True)
    )

    total_sales = detail['sales_value'].sum()
    total_qty = detail['quantity'].sum()

    detail['value_%_of_GT'] = (detail['sales_value'] / total_sales * 100).round(2)
    detail['volume_%_of_GT'] = (detail['quantity'] / total_qty * 100).round(2)
    detail['sales_value'] = detail['sales_value'].round(2)
    detail['quantity'] = detail['quantity'].round(0)

    display(detail)

    # 2) WH vs Others summary
    wh_sales = detail.loc[detail['Final_Outlet_Classification'] == 'WH', 'sales_value'].sum()
    wh_qty = detail.loc[detail['Final_Outlet_Classification'] == 'WH', 'quantity'].sum()
    other_sales = total_sales - wh_sales
    other_qty = total_qty - wh_qty

    wh_vs_others = pd.DataFrame({
        'Group': ['WH (Wholesaler)', 'Others'],
        'sales_value': [round(wh_sales, 2), round(other_sales, 2)],
        'quantity': [round(wh_qty, 0), round(other_qty, 0)],
        'value_%_of_GT': [round((wh_sales / total_sales * 100) if total_sales else 0, 2),
                          round((other_sales / total_sales * 100) if total_sales else 0, 2)],
        'volume_%_of_GT': [round((wh_qty / total_qty * 100) if total_qty else 0, 2),
                           round((other_qty / total_qty * 100) if total_qty else 0, 2)]
    })

    display(wh_vs_others)



=== GT Outlet Classification Mix | INSTA SHAMPOO (STX+STREAX) | 12-ML ===


,Final_Outlet_Classification,sales_value,quantity,value_%_of_GT,volume_%_of_GT
0,WH,50504063.23,4548202.0,54.05,54.29
1,OTHERGT,37226484.17,3326098.0,39.84,39.70
2,SS,5715248.22,503514.0,6.12,6.01


,Group,sales_value,quantity,value_%_of_GT,volume_%_of_GT
0,WH (Wholesaler),50504063.23,4548202.0,54.05,54.29
1,Others,42941732.39,3829612.0,45.95,45.71



=== GT Outlet Classification Mix | INSTA SHAMPOO (STX+STREAX) | 18-ML ===


,Final_Outlet_Classification,sales_value,quantity,value_%_of_GT,volume_%_of_GT
0,WH,1.639683e+08,10813300.0,53.27,53.80
1,OTHERGT,1.278242e+08,8272967.0,41.53,41.16
2,SS,1.601163e+07,1012997.0,5.20,5.04


,Group,sales_value,quantity,value_%_of_GT,volume_%_of_GT
0,WH (Wholesaler),1.639683e+08,10813300.0,53.27,53.8
1,Others,1.438359e+08,9285964.0,46.73,46.2


In [8]:
# Unique outlet count by Final Outlet Classification (independent of product filters)

from pathlib import Path
import pandas as pd
from IPython.display import display

candidate_dirs = [Path('../DATA'), Path('DATA'), Path.cwd() / 'DATA']
data_dir = next((p for p in candidate_dirs if p.exists()), None)
if data_dir is None:
    raise FileNotFoundError("DATA folder not found. Expected ../DATA or DATA")

files = sorted(data_dir.glob('*.parquet'))
if not files:
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

use_cols = ['Outlet_ID', 'Final_Outlet_Classification']
df = pd.concat((pd.read_parquet(f, columns=use_cols) for f in files), ignore_index=True)

df['Outlet_ID'] = df['Outlet_ID'].astype(str).str.strip()
df['Final_Outlet_Classification'] = (
    df['Final_Outlet_Classification']
    .astype(str)
    .str.upper()
    .str.strip()
    .replace({'': 'UNKNOWN', 'NAN': 'UNKNOWN'})
)

outlet_counts = (
    df.drop_duplicates(subset=['Outlet_ID', 'Final_Outlet_Classification'])
      .groupby('Final_Outlet_Classification', as_index=False)
      .agg(unique_outlets=('Outlet_ID', 'nunique'))
      .sort_values('unique_outlets', ascending=False)
      .reset_index(drop=True)
)

total_unique_outlets = df['Outlet_ID'].nunique()
outlet_counts['outlet_%'] = (outlet_counts['unique_outlets'] / total_unique_outlets * 100).round(2)

print(f"Total unique outlets: {total_unique_outlets:,}")
display(outlet_counts)


Total unique outlets: 73,725


,Final_Outlet_Classification,unique_outlets,outlet_%
0,OTHERGT,69600,94.40
1,WH,2846,3.86
2,SS,1861,2.52


In [9]:
from pathlib import Path
import pandas as pd
from IPython.display import display

candidate_dirs = [Path('../DATA'), Path('DATA'), Path.cwd() / 'DATA']
data_dir = next((p for p in candidate_dirs if p.exists()), None)
if data_dir is None:
    raise FileNotFoundError("DATA folder not found. Expected ../DATA or DATA")

files = sorted(data_dir.glob('*.parquet'))
if not files:
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

cols = ['Date', 'Brand', 'Subcategory', 'Sizes', 'MRP', 'Quantity']
df = pd.concat((pd.read_parquet(f, columns=cols) for f in files), ignore_index=True)

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date']).copy()

df['Brand'] = df['Brand'].astype(str).str.upper().str.strip()
df['Subcategory'] = df['Subcategory'].astype(str).str.upper().str.replace(r'\s+', ' ', regex=True).str.strip()
df['Sizes'] = df['Sizes'].astype(str).str.upper().str.replace(' ', '').str.strip()
df['MRP'] = pd.to_numeric(df['MRP'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(0)

scope = df[
    (df['Brand'] == 'STREAX') &
    (df['Subcategory'].isin(['STX INSTA SHAMPOO', 'STREAX INSTA SHAMPOO'])) &
    (df['Sizes'].isin(['12-ML', '18-ML'])) &
    (df['MRP'].notna())
].copy()

scope['Month'] = scope['Date'].dt.to_period('M').dt.to_timestamp()

# Dominant monthly MRP by quantity (robust when multiple MRPs exist in a month)
mrp_qty = (
    scope.groupby(['Sizes', 'Month', 'MRP'], as_index=False)
    .agg(total_qty=('Quantity', 'sum'))
)
idx = mrp_qty.groupby(['Sizes', 'Month'])['total_qty'].idxmax()
monthly_mrp = (
    mrp_qty.loc[idx]
    .sort_values(['Sizes', 'Month'])
    .rename(columns={'MRP': 'dominant_mrp'})
    .reset_index(drop=True)
)

monthly_mrp['prev_mrp'] = monthly_mrp.groupby('Sizes')['dominant_mrp'].shift(1)
monthly_mrp['mrp_change'] = monthly_mrp['dominant_mrp'] - monthly_mrp['prev_mrp']

shifts = monthly_mrp[
    monthly_mrp['prev_mrp'].notna() &
    (monthly_mrp['mrp_change'].abs() > 1e-9)
][['Sizes', 'Month', 'prev_mrp', 'dominant_mrp', 'mrp_change']].reset_index(drop=True)

print("Monthly dominant MRP:")
display(monthly_mrp[['Sizes', 'Month', 'dominant_mrp', 'total_qty']])

print("Detected base price shifts:")
display(shifts if not shifts.empty else pd.DataFrame([{'info': 'No MRP shifts detected'}]))


Monthly dominant MRP:


,Sizes,Month,dominant_mrp,total_qty
0,12-ML,2024-04-01,15.0,245580.0
1,12-ML,2024-05-01,15.0,327035.0
2,12-ML,2024-06-01,15.0,298679.0
3,12-ML,2024-07-01,15.0,292422.0
4,12-ML,2024-08-01,15.0,364412.0
5,12-ML,2024-09-01,15.0,352283.0
6,12-ML,2024-10-01,15.0,312891.0
7,12-ML,2024-11-01,15.0,337189.0
8,12-ML,2024-12-01,15.0,445490.0
9,12-ML,2025-01-01,15.0,393203.0


Detected base price shifts:


,Sizes,Month,prev_mrp,dominant_mrp,mrp_change
0,18-ML,2025-08-01,20.0,22.0,2.0


In [11]:
# Promo rates by FINAL OUTLET CLASSIFICATION (not Outlet_Type)
# Scope: Brand = STREAX, Subcategory = STX+STREAX INSTA SHAMPOO, Sizes = 12-ML & 18-ML
# Formula: avg_discount_% = sum(TotalDiscount) / sum(SalesValue_atBasicRate) * 100

from pathlib import Path
import numpy as np
import pandas as pd
from IPython.display import display

candidate_dirs = [Path('../DATA'), Path('DATA'), Path.cwd() / 'DATA']
data_dir = next((p for p in candidate_dirs if p.exists()), None)
if data_dir is None:
    raise FileNotFoundError("DATA folder not found. Expected ../DATA or DATA")

files = sorted(data_dir.glob('*.parquet'))
if not files:
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

use_cols = [
    'Brand', 'Subcategory', 'Sizes', 'Final_Outlet_Classification',
    'SalesValue_atBasicRate', 'TotalDiscount', 'Quantity'
]
df = pd.concat((pd.read_parquet(f, columns=use_cols) for f in files), ignore_index=True)

# Clean
df['Brand'] = df['Brand'].astype(str).str.upper().str.strip()
df['Subcategory'] = df['Subcategory'].astype(str).str.upper().str.replace(r'\s+', ' ', regex=True).str.strip()
df['Sizes'] = df['Sizes'].astype(str).str.upper().str.replace(' ', '').str.strip()
df['Final_Outlet_Classification'] = (
    df['Final_Outlet_Classification']
    .astype(str)
    .str.upper()
    .str.strip()
    .replace({'': 'UNKNOWN', 'NAN': 'UNKNOWN'})
)

df['SalesValue_atBasicRate'] = pd.to_numeric(df['SalesValue_atBasicRate'], errors='coerce').fillna(0.0)
df['TotalDiscount'] = pd.to_numeric(df['TotalDiscount'], errors='coerce').fillna(0.0)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(0.0)

scope = df[
    (df['Brand'] == 'STREAX') &
    (df['Subcategory'].isin(['STX INSTA SHAMPOO', 'STREAX INSTA SHAMPOO'])) &
    (df['Sizes'].isin(['12-ML', '18-ML']))
].copy()

# 1) Combined (12+18) promo rate by final outlet classification
foc_summary = (
    scope.groupby('Final_Outlet_Classification', as_index=False)
    .agg(
        sales_value=('SalesValue_atBasicRate', 'sum'),
        total_discount=('TotalDiscount', 'sum'),
        quantity=('Quantity', 'sum')
    )
)

foc_summary['avg_discount_%'] = np.where(
    foc_summary['sales_value'] != 0,
    (foc_summary['total_discount'] / foc_summary['sales_value']) * 100,
    np.nan
)

foc_summary = foc_summary.sort_values('sales_value', ascending=False).reset_index(drop=True)
foc_summary['sales_value'] = foc_summary['sales_value'].round(2)
foc_summary['total_discount'] = foc_summary['total_discount'].round(2)
foc_summary['quantity'] = foc_summary['quantity'].round(0)
foc_summary['avg_discount_%'] = foc_summary['avg_discount_%'].round(2)

print("Promo rate by Final Outlet Classification (combined 12-ML + 18-ML):")
display(foc_summary)

# 2) Split by size (12 vs 18) within final outlet classification
foc_size_summary = (
    scope.groupby(['Sizes', 'Final_Outlet_Classification'], as_index=False)
    .agg(
        sales_value=('SalesValue_atBasicRate', 'sum'),
        total_discount=('TotalDiscount', 'sum'),
        quantity=('Quantity', 'sum')
    )
)

foc_size_summary['avg_discount_%'] = np.where(
    foc_size_summary['sales_value'] != 0,
    (foc_size_summary['total_discount'] / foc_size_summary['sales_value']) * 100,
    np.nan
)

foc_size_summary = foc_size_summary.sort_values(['Sizes', 'sales_value'], ascending=[True, False]).reset_index(drop=True)
foc_size_summary['sales_value'] = foc_size_summary['sales_value'].round(2)
foc_size_summary['total_discount'] = foc_size_summary['total_discount'].round(2)
foc_size_summary['quantity'] = foc_size_summary['quantity'].round(0)
foc_size_summary['avg_discount_%'] = foc_size_summary['avg_discount_%'].round(2)

print("Promo rate by Final Outlet Classification (split by size):")
display(foc_size_summary)


Promo rate by Final Outlet Classification (combined 12-ML + 18-ML):


,Final_Outlet_Classification,sales_value,total_discount,quantity,avg_discount_%
0,WH,2.144724e+08,37194815.13,15361502.0,17.34
1,OTHERGT,1.650507e+08,23097349.34,11599065.0,13.99
2,SS,2.172688e+07,3303596.58,1516511.0,15.21


Promo rate by Final Outlet Classification (split by size):


,Sizes,Final_Outlet_Classification,sales_value,total_discount,quantity,avg_discount_%
0,12-ML,WH,5.050406e+07,9519173.89,4548202.0,18.85
1,12-ML,OTHERGT,3.722648e+07,5785186.33,3326098.0,15.54
2,12-ML,SS,5.715248e+06,855816.15,503514.0,14.97
3,18-ML,WH,1.639683e+08,27675641.24,10813300.0,16.88
4,18-ML,OTHERGT,1.278242e+08,17312163.01,8272967.0,13.54
5,18-ML,SS,1.601163e+07,2447780.43,1012997.0,15.29


In [13]:
# Trend + Seasonality in Plotly
# Scope: Brand=STREAX, Subcategory in {STX INSTA SHAMPOO, STREAX INSTA SHAMPOO}, Sizes in {12-ML, 18-ML}
# Metrics: SalesValue_atBasicRate, Quantity

from pathlib import Path
import numpy as np
import pandas as pd
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots

try:
    from scipy.stats import theilslopes
    HAS_SCIPY = True
except Exception:
    HAS_SCIPY = False

# ---------- Load ----------
candidate_dirs = [Path('../DATA'), Path('DATA'), Path.cwd() / 'DATA']
data_dir = next((p for p in candidate_dirs if p.exists()), None)
if data_dir is None:
    raise FileNotFoundError("DATA folder not found. Expected ../DATA or DATA")

files = sorted(data_dir.glob('*.parquet'))
if not files:
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

use_cols = ['Date', 'Brand', 'Subcategory', 'Sizes', 'SalesValue_atBasicRate', 'Quantity']
df = pd.concat((pd.read_parquet(f, columns=use_cols) for f in files), ignore_index=True)

# ---------- Clean ----------
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date']).copy()
df['Brand'] = df['Brand'].astype(str).str.upper().str.strip()
df['Subcategory'] = df['Subcategory'].astype(str).str.upper().str.replace(r'\s+', ' ', regex=True).str.strip()
df['Sizes'] = df['Sizes'].astype(str).str.upper().str.replace(' ', '').str.strip()
df['SalesValue_atBasicRate'] = pd.to_numeric(df['SalesValue_atBasicRate'], errors='coerce').fillna(0.0)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(0.0)

scope = df[
    (df['Brand'] == 'STREAX') &
    (df['Subcategory'].isin(['STX INSTA SHAMPOO', 'STREAX INSTA SHAMPOO'])) &
    (df['Sizes'].isin(['12-ML', '18-ML']))
].copy()

scope['Month'] = scope['Date'].dt.to_period('M').dt.to_timestamp()

monthly = (
    scope.groupby(['Sizes', 'Month'], as_index=False)
    .agg(
        sales_value=('SalesValue_atBasicRate', 'sum'),
        quantity=('Quantity', 'sum')
    )
    .sort_values(['Sizes', 'Month'])
)

def fit_trend_seasonality(sub_df, y_col):
    mdf = sub_df[['Month', y_col]].copy().sort_values('Month').reset_index(drop=True)
    y = mdf[y_col].to_numpy(dtype=float)
    y = np.clip(y, 1e-9, None)

    n = len(mdf)
    t = np.arange(n, dtype=float)
    mon = mdf['Month'].dt.month

    mon_cat = pd.Categorical(mon, categories=list(range(1, 13)), ordered=True)
    dummies = pd.get_dummies(mon_cat, drop_first=True, dtype=float)  # months 2..12
    X = np.column_stack([np.ones(n), t, dummies.to_numpy()])

    y_log = np.log(y)
    beta, *_ = np.linalg.lstsq(X, y_log, rcond=None)
    yhat_log = X @ beta
    yhat = np.exp(yhat_log)

    ss_res = np.sum((y_log - yhat_log) ** 2)
    ss_tot = np.sum((y_log - y_log.mean()) ** 2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else np.nan

    slope_log = beta[1]
    monthly_growth_pct = (np.exp(slope_log) - 1) * 100
    annualized_growth_pct = (np.exp(12 * slope_log) - 1) * 100

    if HAS_SCIPY and n >= 3:
        ts_slope, _, _, _ = theilslopes(y_log, t, 0.95)
        ts_monthly_growth_pct = (np.exp(ts_slope) - 1) * 100
    else:
        ts_monthly_growth_pct = np.nan

    month_effect = {1: 0.0}
    for m in range(2, 13):
        month_effect[m] = beta[1 + (m - 1)]

    observed_months = sorted(mdf['Month'].dt.month.unique().tolist())
    raw = {m: np.exp(month_effect[m]) for m in observed_months}
    norm = np.mean(list(raw.values()))
    season_idx = {m: (raw[m] / norm if norm > 0 else np.nan) for m in observed_months}

    seasonality_table = pd.DataFrame({
        'month_num': observed_months,
        'month_name': [pd.Timestamp(2000, m, 1).strftime('%b') for m in observed_months],
        'seasonality_index': [season_idx[m] for m in observed_months]
    }).sort_values('month_num')

    trend_summary = {
        'n_months': n,
        'start_month': mdf['Month'].min(),
        'end_month': mdf['Month'].max(),
        'monthly_growth_%': monthly_growth_pct,
        'annualized_growth_%': annualized_growth_pct,
        'r2_log_model': r2,
        'theilsen_monthly_growth_%': ts_monthly_growth_pct
    }

    plot_df = mdf.copy()
    plot_df['actual'] = y
    plot_df['fitted'] = yhat

    return trend_summary, seasonality_table, plot_df

summaries = []

for size in ['12-ML', '18-ML']:
    part = monthly[monthly['Sizes'] == size].copy()

    for metric_col, metric_label in [('sales_value', 'Sales Value'), ('quantity', 'Quantity')]:
        trend_summary, seasonality_table, plot_df = fit_trend_seasonality(part, metric_col)

        summaries.append({
            'size': size,
            'metric': metric_label,
            **trend_summary
        })

        title_text = (
            f"{size} | {metric_label} | "
            f"Monthly trend={trend_summary['monthly_growth_%']:.2f}% | "
            f"Annualized={trend_summary['annualized_growth_%']:.2f}% | "
            f"R2={trend_summary['r2_log_model']:.3f}"
        )

        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=("Actual vs Fitted (Trend + Seasonality)", "Seasonality Index"),
            horizontal_spacing=0.12
        )

        fig.add_trace(
            go.Scatter(
                x=plot_df['Month'], y=plot_df['actual'],
                mode='lines+markers', name='Actual',
                line=dict(width=2)
            ),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(
                x=plot_df['Month'], y=plot_df['fitted'],
                mode='lines+markers', name='Fitted',
                line=dict(width=2, dash='dot')
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Bar(
                x=seasonality_table['month_name'],
                y=seasonality_table['seasonality_index'],
                name='Seasonality Index'
            ),
            row=1, col=2
        )
        fig.add_hline(
            y=1.0, line_dash='dash', line_color='red',
            annotation_text='Average=1.0', annotation_position='top right',
            row=1, col=2
        )

        fig.update_layout(
            title=title_text,
            template='plotly_white',
            height=450,
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0.0)
        )

        fig.update_xaxes(title_text='Month', row=1, col=1)
        fig.update_yaxes(title_text=metric_label, row=1, col=1)
        fig.update_xaxes(title_text='Calendar Month', row=1, col=2)
        fig.update_yaxes(title_text='Index', row=1, col=2)

        fig.show()

        print(
            f"{size} | {metric_label} -> "
            f"Monthly={trend_summary['monthly_growth_%']:.2f}%, "
            f"Annualized={trend_summary['annualized_growth_%']:.2f}%, "
            f"R2={trend_summary['r2_log_model']:.3f}, "
            f"Theil-Sen={trend_summary['theilsen_monthly_growth_%']:.2f}%"
        )
        display(seasonality_table.assign(seasonality_index=seasonality_table['seasonality_index'].round(3)))

summary_df = pd.DataFrame(summaries)
summary_df['monthly_growth_%'] = summary_df['monthly_growth_%'].round(2)
summary_df['annualized_growth_%'] = summary_df['annualized_growth_%'].round(2)
summary_df['r2_log_model'] = summary_df['r2_log_model'].round(3)
summary_df['theilsen_monthly_growth_%'] = summary_df['theilsen_monthly_growth_%'].round(2)

print("\nFinal Trend Summary:")
display(summary_df)


12-ML | Sales Value -> Monthly=4.47%, Annualized=69.06%, R2=0.614, Theil-Sen=3.55%


,month_num,month_name,seasonality_index
0,1,Jan,0.896
1,2,Feb,0.951
2,3,Mar,1.186
3,4,Apr,1.371
4,5,May,0.814
5,6,Jun,0.895
6,7,Jul,1.181
7,8,Aug,0.801
8,9,Sep,1.292
9,10,Oct,0.705


12-ML | Quantity -> Monthly=4.27%, Annualized=65.09%, R2=0.605, Theil-Sen=4.01%


,month_num,month_name,seasonality_index
0,1,Jan,0.949
1,2,Feb,0.927
2,3,Mar,1.160
3,4,Apr,1.328
4,5,May,0.791
5,6,Jun,0.870
6,7,Jul,1.150
7,8,Aug,0.785
8,9,Sep,1.315
9,10,Oct,0.719


18-ML | Sales Value -> Monthly=1.62%, Annualized=21.23%, R2=0.832, Theil-Sen=2.03%


,month_num,month_name,seasonality_index
0,1,Jan,1.000
1,2,Feb,0.965
2,3,Mar,1.404
3,4,Apr,0.760
4,5,May,1.384
5,6,Jun,0.889
6,7,Jul,1.018
7,8,Aug,0.960
8,9,Sep,0.823
9,10,Oct,0.948


18-ML | Quantity -> Monthly=0.75%, Annualized=9.36%, R2=0.834, Theil-Sen=1.27%


,month_num,month_name,seasonality_index
0,1,Jan,1.057
1,2,Feb,0.974
2,3,Mar,1.429
3,4,Apr,0.741
4,5,May,1.360
5,6,Jun,0.859
6,7,Jul,0.988
7,8,Aug,0.921
8,9,Sep,0.814
9,10,Oct,0.948



Final Trend Summary:


,size,metric,n_months,start_month,end_month,monthly_growth_%,annualized_growth_%,r2_log_model,theilsen_monthly_growth_%
0,12-ML,Sales Value,19,2024-04-01,2025-10-01,4.47,69.06,0.614,3.55
1,12-ML,Quantity,19,2024-04-01,2025-10-01,4.27,65.09,0.605,4.01
2,18-ML,Sales Value,19,2024-04-01,2025-10-01,1.62,21.23,0.832,2.03
3,18-ML,Quantity,19,2024-04-01,2025-10-01,0.75,9.36,0.834,1.27


In [18]:
# Month-wise subcategories inside Category = "STREAX"

from pathlib import Path
import pandas as pd
from IPython.display import display

# 1) Load parquet files
candidate_dirs = [Path("../DATA"), Path("DATA"), Path.cwd() / "DATA"]
data_dir = next((p for p in candidate_dirs if p.exists()), None)
if data_dir is None:
    raise FileNotFoundError("DATA folder not found (checked ../DATA and DATA)")

files = sorted(data_dir.glob("*.parquet"))
if not files:
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

use_cols = ["Date", "Category", "Subcategory", "SalesValue_atBasicRate", "Quantity"]
df = pd.concat((pd.read_parquet(f, columns=use_cols) for f in files), ignore_index=True)

# 2) Clean + filter to STREAX category
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df = df.dropna(subset=["Date"]).copy()
df["Category"] = df["Category"].astype(str).str.strip()
df["Subcategory"] = df["Subcategory"].astype(str).str.strip()

streax = df[df["Category"].str.upper().eq("STREAX")].copy()
streax["Month"] = streax["Date"].dt.to_period("M").astype(str)

# 3) Output A: each month + all distinct subcategories
month_subcat_list = (
    streax.groupby("Month")["Subcategory"]
    .apply(lambda s: sorted(set(x for x in s if x)))
    .reset_index(name="Subcategories")
)
month_subcat_list["Subcategory_Count"] = month_subcat_list["Subcategories"].str.len()

print("Month-wise distinct subcategories (Category=STREAX):")
display(month_subcat_list[["Month", "Subcategory_Count", "Subcategories"]])

# 4) Output B: month x subcategory metrics
month_subcat_metrics = (
    streax.groupby(["Month", "Subcategory"], as_index=False)
    .agg(
        Sales_Value=("SalesValue_atBasicRate", "sum"),
        Quantity=("Quantity", "sum"),
        Rows=("Subcategory", "size"),
    )
    .sort_values(["Month", "Sales_Value"], ascending=[True, False])
)

print("\nMonth x Subcategory metrics:")
display(month_subcat_metrics)

# 5) Optional: presence matrix (1 if subcategory exists in that month)
presence_matrix = (
    month_subcat_metrics.assign(Present=1)
    .pivot(index="Month", columns="Subcategory", values="Present")
    .fillna(0)
    .astype(int)
    .sort_index()
)

print("\nPresence matrix (Month x Subcategory):")
display(presence_matrix)

presence_matrix.to_csv("subcategory.csv")



Month-wise distinct subcategories (Category=STREAX):


,Month,Subcategory_Count,Subcategories
0,2024-04,13,"[SHAMPOO and CONDITIONER, STRX SMART MINI HAIR..."
1,2024-05,15,"[SHAMPOO and CONDITIONER, STREAX HEAT PROTECT ..."
2,2024-06,14,"[SHAMPOO and CONDITIONER, STRX SMART MINI HAIR..."
3,2024-07,13,"[SHAMPOO and CONDITIONER, STRX SMART MINI HAIR..."
4,2024-08,13,"[SHAMPOO and CONDITIONER, STREAX HEAT PROTECT ..."
5,2024-09,15,"[GEL HAIR COLOUR, SHAMPOO and CONDITIONER, STR..."
6,2024-10,14,"[GEL HAIR COLOUR, SHAMPOO and CONDITIONER, STR..."
7,2024-11,13,"[GEL HAIR COLOUR, SHAMPOO and CONDITIONER, STR..."
8,2024-12,14,"[GEL HAIR COLOUR, SHAMPOO and CONDITIONER, STR..."
9,2025-01,13,"[GEL HAIR COLOUR, SHAMPOO and CONDITIONER, STR..."



Month x Subcategory metrics:


,Month,Subcategory,Sales_Value,Quantity,Rows
7,2024-04,STX INSTA SHAMPOO,17928287.91,1246242.0,53511
11,2024-04,STX WLNT SERUM,11370968.59,264638.0,24995
2,2024-04,STX HAIR HIGHLIGHTING,2106551.96,14074.0,5978
4,2024-04,STX HC MINI,2081093.58,36670.0,16218
3,2024-04,STX HC 120,1436422.25,11621.0,6887
...,...,...,...,...,...
308,2025-11,STREAX ROSEMARY WATER,22022.36,218.0,52
322,2025-11,STX YOUTHFUL SHINE FACE CREAM,21116.00,100.0,38
319,2025-11,STX SUNSCREEN STICK 20 GM,2471.80,10.0,4
320,2025-11,STX ULTRA LIGHT,575.04,4.0,3



Presence matrix (Month x Subcategory):


Subcategory,FLORO FACE WASH,GEL HAIR COLOUR,SERUM SHINE MASQUE,SHAMPOO and CONDITIONER,STREAX CRAFT,STREAX FACE CREAM,STREAX FACE WASH,STREAX HEAT PROTECT SPARY,STREAX INSTA NATURALS HC,STREAX INSTA SHAMPOO,...,STX SERUM SHINE SHAMPOO,STX SUNSCREEN STICK 20 GM,STX TLC MINI,STX TLC REG,STX TOUCH UP,STX ULTRA LIGHT,STX ULTRA LIGHT.,STX WLNT SERUM,STX YOUTHFUL SHINE FACE CREAM,Streax Hair Serum
Month,,,,,,,,,,,,,,,,,,,,,
2024-04,0,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,1,0,1,0,1
2024-05,0,0,0,1,0,0,0,1,0,0,...,0,0,1,1,1,1,0,1,0,1
2024-06,0,0,0,1,0,0,0,0,0,0,...,0,0,1,1,1,0,1,1,0,1
2024-07,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
2024-08,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,1,0,0,1,0,1
2024-09,0,1,0,1,0,0,0,1,0,0,...,0,0,1,0,1,1,0,1,0,1
2024-10,0,1,0,1,0,0,0,1,0,0,...,0,0,1,0,1,0,0,1,0,1
2024-11,0,1,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,1
2024-12,0,1,0,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,1,0,1


In [24]:
# Absolute sales and quantity (MH, UP, TOTAL)
# For combined STX + STREAX INSTA SHAMPOO
# Split by size: 12-ML and 18-ML
# For FY25 and FY26 (FY26 till Nov based on available data)
# Saves CSV

from pathlib import Path
import pandas as pd
import numpy as np

# 1) Load data
candidate_dirs = [Path('../DATA'), Path('DATA'), Path.cwd() / 'DATA']
data_dir = next((p for p in candidate_dirs if p.exists()), None)
if data_dir is None:
    raise FileNotFoundError("DATA folder not found. Expected ../DATA or DATA")

files = sorted(data_dir.glob('*.parquet'))
if not files:
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

use_cols = ['Date', 'Final_State', 'Subcategory', 'Sizes', 'SalesValue_atBasicRate', 'Quantity']
df = pd.concat((pd.read_parquet(f, columns=use_cols) for f in files), ignore_index=True)

# 2) Clean/normalize
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date']).copy()

df['Final_State'] = df['Final_State'].astype(str).str.upper().str.strip()
df['Subcategory'] = (
    df['Subcategory'].astype(str).str.upper().str.replace(r'\s+', ' ', regex=True).str.strip()
)
df['Sizes'] = (
    df['Sizes'].astype(str).str.upper().str.replace(' ', '').str.strip()
)

# 3) Keep combined target product + target sizes
target_subcats = {'STX INSTA SHAMPOO', 'STREAX INSTA SHAMPOO'}
target_sizes = {'12-ML', '18-ML'}

scope = df[
    df['Subcategory'].isin(target_subcats) &
    df['Sizes'].isin(target_sizes)
].copy()

# 4) FY mapping (Apr-Mar)
scope['fy'] = np.where(scope['Date'].dt.month >= 4, scope['Date'].dt.year + 1, scope['Date'].dt.year)
scope['fy_label'] = 'FY' + scope['fy'].astype(str).str[-2:]

# Keep only FY25 and FY26
scope = scope[scope['fy_label'].isin(['FY25', 'FY26'])].copy()

# FY26 till Nov only (Apr-Nov 2025)
scope = scope[
    ~(
        (scope['fy_label'] == 'FY26') &
        (scope['Date'].dt.year == 2025) &
        (scope['Date'].dt.month > 11)
    )
].copy()

# 5) Map states to MH / UP / OTHERS
scope['region'] = np.where(scope['Final_State'].eq('MAH'), 'MH',
                    np.where(scope['Final_State'].eq('UP'), 'UP', 'OTHERS'))

# 6) Aggregate by FY + Size + Region
by_region = (
    scope.groupby(['fy_label', 'Sizes', 'region'], as_index=False)
         .agg(
             sales_value=('SalesValue_atBasicRate', 'sum'),
             quantity=('Quantity', 'sum')
         )
)

# 7) Pivot so MH/UP/TOTAL are columns
sales_piv = by_region.pivot_table(
    index=['fy_label', 'Sizes'],
    columns='region',
    values='sales_value',
    aggfunc='sum',
    fill_value=0.0
).reset_index()

qty_piv = by_region.pivot_table(
    index=['fy_label', 'Sizes'],
    columns='region',
    values='quantity',
    aggfunc='sum',
    fill_value=0.0
).reset_index()

# Ensure columns exist
for c in ['MH', 'UP']:
    if c not in sales_piv.columns:
        sales_piv[c] = 0.0
    if c not in qty_piv.columns:
        qty_piv[c] = 0.0

# TOTAL from full scope (not only MH+UP)
total_vals = (
    scope.groupby(['fy_label', 'Sizes'], as_index=False)
         .agg(
             sales_total=('SalesValue_atBasicRate', 'sum'),
             qty_total=('Quantity', 'sum')
         )
)

out = sales_piv.merge(
    qty_piv,
    on=['fy_label', 'Sizes'],
    suffixes=('_sales', '_qty')
).merge(
    total_vals,
    on=['fy_label', 'Sizes'],
    how='left'
)

# Rename/select final columns
final = pd.DataFrame({
    'FY': out['fy_label'],
    'SKU_Size': out['Sizes'],
    'MH_Sales': out.get('MH_sales', 0.0),
    'UP_Sales': out.get('UP_sales', 0.0),
    'Total_Sales': out['sales_total'],
    'MH_Quantity': out.get('MH_qty', 0.0),
    'UP_Quantity': out.get('UP_qty', 0.0),
    'Total_Quantity': out['qty_total'],
})

# Order rows
fy_order = {'FY25': 0, 'FY26': 1}
size_order = {'12-ML': 0, '18-ML': 1}
final['__fy_order'] = final['FY'].map(fy_order).fillna(99)
final['__size_order'] = final['SKU_Size'].map(size_order).fillna(99)
final = final.sort_values(['__fy_order', '__size_order']).drop(columns=['__fy_order', '__size_order']).reset_index(drop=True)

# Round format
for c in ['MH_Sales', 'UP_Sales', 'Total_Sales']:
    final[c] = final[c].round(2)
for c in ['MH_Quantity', 'UP_Quantity', 'Total_Quantity']:
    final[c] = final[c].round(0)

# 8) Save CSV
out_file = Path("insta_shampoo_abs_mh_up_total_fy25_fy26_till_nov.csv")
final.to_csv(out_file, index=False)

print(final)
print(f"\nSaved: {out_file.resolve()}")


     FY SKU_Size      MH_Sales     UP_Sales   Total_Sales  MH_Quantity  \
0  FY25    12-ML  2.545658e+07  21612605.35  4.706918e+07    2316703.0   
1  FY25    18-ML  1.124230e+08  72626022.50  1.850490e+08    7611086.0   
2  FY26    12-ML  2.881232e+07  28826503.89  5.763882e+07    2539151.0   
3  FY26    18-ML  8.564842e+07  51091977.93  1.367404e+08    5252761.0   

   UP_Quantity  Total_Quantity  
0    1974976.0       4291679.0  
1    4956213.0      12567299.0  
2    2539417.0       5078568.0  
3    3128970.0       8381731.0  

Saved: C:\Users\abqua\Desktop\HRI app\jupyter\insta_shampoo_abs_mh_up_total_fy25_fy26_till_nov.csv


In [25]:
# Contribution of WH in total sales, by Slab (1-4) and Size (12-ML, 18-ML)
# Product scope: combined STX INSTA SHAMPOO + STREAX INSTA SHAMPOO
# Uses Final_Outlet_Classification
# Saves CSV

from pathlib import Path
import pandas as pd
import numpy as np

candidate_dirs = [Path('../DATA'), Path('DATA'), Path.cwd() / 'DATA']
data_dir = next((p for p in candidate_dirs if p.exists()), None)
if data_dir is None:
    raise FileNotFoundError("DATA folder not found. Expected ../DATA or DATA")

files = sorted(data_dir.glob('*.parquet'))
if not files:
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

use_cols = ['Subcategory', 'Sizes', 'Slab', 'Final_Outlet_Classification', 'SalesValue_atBasicRate']
df = pd.concat((pd.read_parquet(f, columns=use_cols) for f in files), ignore_index=True)

# Normalize
df['Subcategory'] = (
    df['Subcategory'].astype(str).str.upper().str.replace(r'\s+', ' ', regex=True).str.strip()
)
df['Sizes'] = (
    df['Sizes'].astype(str).str.upper().str.replace(' ', '').str.strip()
)
df['Slab'] = df['Slab'].astype(str).str.lower().str.strip()
df['Final_Outlet_Classification'] = (
    df['Final_Outlet_Classification'].astype(str).str.upper().str.replace(' ', '').str.strip()
)

# Scope: combined subcategories + sizes + slabs 1-4
target_subcats = {'STX INSTA SHAMPOO', 'STREAX INSTA SHAMPOO'}
target_sizes = {'12-ML', '18-ML'}
target_slabs = {'slab1', 'slab2', 'slab3', 'slab4'}

scope = df[
    df['Subcategory'].isin(target_subcats) &
    df['Sizes'].isin(target_sizes) &
    df['Slab'].isin(target_slabs)
].copy()

# Total sales per size+slab
totals = (
    scope.groupby(['Sizes', 'Slab'], as_index=False)
         .agg(total_sales=('SalesValue_atBasicRate', 'sum'))
)

# WH sales per size+slab
wh_sales = (
    scope[scope['Final_Outlet_Classification'].eq('WH')]
    .groupby(['Sizes', 'Slab'], as_index=False)
    .agg(wh_sales=('SalesValue_atBasicRate', 'sum'))
)

out = totals.merge(wh_sales, on=['Sizes', 'Slab'], how='left')
out['wh_sales'] = out['wh_sales'].fillna(0.0)

out['wh_contribution_%'] = np.where(
    out['total_sales'] != 0,
    out['wh_sales'] / out['total_sales'] * 100.0,
    np.nan
)

# Clean labels and ordering
out['Slab'] = out['Slab'].str.replace('slab', 'slab', regex=False)
out['total_sales'] = out['total_sales'].round(2)
out['wh_sales'] = out['wh_sales'].round(2)
out['wh_contribution_%'] = out['wh_contribution_%'].round(2)

size_order = {'12-ML': 0, '18-ML': 1}
slab_order = {'slab1': 1, 'slab2': 2, 'slab3': 3, 'slab4': 4}
out['__size_order'] = out['Sizes'].map(size_order).fillna(99)
out['__slab_order'] = out['Slab'].map(slab_order).fillna(99)

final = out.sort_values(['__size_order', '__slab_order']).drop(columns=['__size_order', '__slab_order']).reset_index(drop=True)
final = final.rename(columns={'Sizes': 'SKU_Size'})

print(final)

out_file = Path("wh_contribution_by_slab_size_insta_shampoo.csv")
final.to_csv(out_file, index=False)
print(f"\nSaved: {out_file.resolve()}")


  SKU_Size   Slab   total_sales     wh_sales  wh_contribution_%
0    12-ML  slab1  2.928021e+07    694027.86               2.37
1    12-ML  slab2  3.359371e+07  20647520.76              61.46
2    12-ML  slab3  1.440211e+07  11744481.98              81.55
3    12-ML  slab4  2.734896e+07  23896115.59              87.37
4    18-ML  slab1  1.064417e+08   1380241.02               1.30
5    18-ML  slab2  6.231190e+07  39150770.66              62.83
6    18-ML  slab3  4.380528e+07  37187051.58              84.89
7    18-ML  slab4  1.087772e+08  91853183.37              84.44

Saved: C:\Users\abqua\Desktop\HRI app\jupyter\wh_contribution_by_slab_size_insta_shampoo.csv
